In [1]:
import sys
import os
from pathlib import Path 
from torchvision.transforms import v2
import torch
import matplotlib.pyplot as plt
import pickle
modulepath = Path.cwd().parent / "utils"
modulepath = str(modulepath)
projectpath = Path.cwd().parent
projectpath = str(projectpath)
if modulepath not in sys.path:
    sys.path.insert(0,str(modulepath))
if projectpath not in sys.path:
    sys.path.insert(0,str(projectpath))  
print(sys.path)


['c:\\Users\\oriol\\GitHub_Repos\\CNN-classification', 'c:\\Users\\oriol\\GitHub_Repos\\CNN-classification\\utils', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\python312.zip', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\DLLs', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\Lib', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126', '', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\Lib\\site-packages', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\Lib\\site-packages\\win32', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch_cu126\\Lib\\site-packages\\Pythonwin']


In [ ]:
from utils.data_loaders import create_train_test_dataloaders
transform = v2.Compose([v2.RandomCrop(size=(20, 20))])#,v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])


In [ ]:
train_basic,test_basic = create_train_test_dataloaders("FashionMNIST")
train_augmented,test_augmented = create_train_test_dataloaders("FashionMNIST",transform=transform)
img_basic,label_basic = next(iter(train_basic))
img_augmented,label_augmented = next(iter(train_augmented))

In [ ]:
train_basic,test_basic = create_train_test_dataloaders("some spaces are good")

print(train_basic.dataset.root.split("\\")[-1])
name = os.path.split(os.path.normpath(train_basic.dataset.root))[-1]
print(name)

In [ ]:
fig,ax = plt.subplots(1,2)
ax[0].imshow(img_basic[0].permute(1,2,0),cmap="gray")
ax[1].imshow(img_augmented[0].permute(1,2,0),cmap="gray")
ax[0].axis(False)
ax[1].axis(False)

In [ ]:
img_basic.shape

In [ ]:
from utils.model_architectures import TinyVGG_1
from torchinfo import summary 

tvgg = TinyVGG_1(1,20,10)

In [ ]:
summary(tvgg,input_size=[32,1,28,28])

In [ ]:
tvgg.cpu()
tvgg.forward(img_basic[0].unsqueeze(dim=0))
torch.softmax(tvgg.forward(img_basic[0].unsqueeze(dim=0)),1)

#next(tvgg.parameters()).is_cuda #check if model on cuda

In [ ]:
from utils.engine import train
from utils.model_architectures import TinyVGG_1
from utils.data_loaders import create_train_test_dataloaders
from torch import nn
from utils.helpers import create_writer
from utils.helpers import create_experiment_metadata
train_dataloader, test_dataloader = create_train_test_dataloaders("some spaces are good")

device = "cuda" if torch.cuda.is_available() else "cpu"

model_0 = TinyVGG_1(input_shape=1,hidden_channels=10,output_shape=10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),lr=0.01)
#print(f"Is the model on GPU {next(model_0.parameters()).is_cuda}")

model_name = model_0._get_name()
experiment_name = "Test_writer_and_metadata"
extra = "3 epochs SDG"
NUM_EPOCHS = 3

writer = create_writer(experiment_name=experiment_name,model_name=model_name,extra=extra)
#create_experiment_metadata(writer=writer,train_dataloader=train_dataloader,test_dataloader=test_dataloader,model=model_0,optimizer=optimizer,loss_fn=loss_fn,epochs=NUM_EPOCHS)

train(model=model_0,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer,loss_fn=loss_fn,epochs=NUM_EPOCHS,writer=writer)


[INFO] Created SummaryWriter, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\runs\TinyVGG_1\Test_writer_and_metadata\3 epochs SDG...
[INFO] Created experiment metadata, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\runs_metadata\TinyVGG_1\Test_writer_and_metadata\3 epochs SDG...


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
with open(r"C:\Users\oriol\GitHub_Repos\CNN-classification\runs_metadata\TinyVGG_1\Test_writer_and_metadata\3 epochs SDG\Metadata.pkl", 'rb') as file:
    meta_dict = pickle.load(file)

In [2]:
from utils.config import RUNS_METADATA_DIRECTORY
from utils.config import METADATA_FILENAME

def retrieve_metadata(model_name:str, experiment_name:str, extra=None):
    """AAA"""
    if extra:
        metadata_path = os.path.join(RUNS_METADATA_DIRECTORY, model_name, 
                               experiment_name, extra, METADATA_FILENAME)
    else:
        metadata_path = os.path.join(RUNS_METADATA_DIRECTORY, model_name, 
                               experiment_name, METADATA_FILENAME)
    if os.path.exists(metadata_path):
        with open(metadata_path, 'rb') as file:
            metadata = pickle.load(file)
        return metadata
    else:
        print(f"No metadata stored in {metadata_path}")

In [ ]:

meta_dict = retrieve_metadata(model_name="TinyVGG_1", experiment_name="Test_writer_and_metadata", extra="None"):

{'layer_1': Sequential(
   (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
   (3): ReLU()
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 'layer_2': Sequential(
   (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
   (3): ReLU()
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 'layer_3': Sequential(
   (0): Flatten(start_dim=1, end_dim=-1)
   (1): Linear(in_features=160, out_features=10, bias=True)
 )}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
from utils.engine import train
from utils.model_architectures import TinyVGG_1
from utils.data_loaders import create_train_test_dataloaders
from torch import nn
from utils.helpers import create_writer
from utils.helpers import create_experiment_metadata
train_dataloader, test_dataloader = create_train_test_dataloaders()

device = "cuda" if torch.cuda.is_available() else "cpu"

model_0 = TinyVGG_1(input_shape=1,hidden_channels=10,output_shape=10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),lr=0.01)

In [ ]:
optimizer.__getstate__()['defaults']

In [ ]:
model_0.__getstate__()['_modules']

In [ ]:
import pickle
with open(r"C:\Users\oriol\GitHub_Repos\CNN-classification\runs_metadata\TinyVGG_1\Test_writer_and_dict\2 epochs\Metadata.pkl", 'rb') as file:
    meta_dict = pickle.load(file)

In [ ]:
print(meta_dict)

In [ ]:
wrtr.__getstate__()['log_dir']

In [ ]:
import os
from utils.config import RUNS_DIRECTORY
from utils.helpers import create_writer
wrtr,dctdr = create_writer(model_name="None",experiment_name="Test expname extraction",extra="Please")

relpath = os.path.normpath(os.path.relpath(wrtr.__getstate__()['log_dir'],RUNS_DIRECTORY))
components = relpath.split(os.sep)

In [ ]:
print(f"Model name '{components[0]}' and experiment name '{components[1]}'")
type(components[0])

In [ ]:
from utils.model_architectures import TinyVGG_1

device = "cuda" if torch.cuda.is_available() else "cpu"

model_0 = TinyVGG_1(input_shape=1,hidden_channels=10,output_shape=10).to(device)

In [ ]:
from utils.config import RUNS_METADATA_DIRECTORY
dictpath = os.path.join(RUNS_METADATA_DIRECTORY,relpath)
print(dictpath)

In [ ]:
import os
from utils.config import RUNS_METADATA_DIRECTORY
import pickle
metadata_file = os.path.join(RUNS_METADATA_DIRECTORY,"TinyVGG_1","Test_writer_and_newdict","1 epochs","Metadata.pkl")
with open(metadata_file, 'rb') as f:
    resdict = pickle.load(f)

In [ ]:
resdict